In [1]:
from qiskit import QuantumCircuit
import numpy as np
import math

# 1) MATRICE D'IMAGE (grayscale)

In [2]:
img = np.array([
    [25, 100],
    [180, 230]
], dtype=float)


# 2)Transformation pixel → angle θ = (π/2)*(pixel/255)

In [3]:
theta = (np.pi/2) * (img.flatten() / 255.0)
print("Angles theta FRQI :", theta)

Angles theta FRQI : [0.15399964 0.61599856 1.10879741 1.41679669]


# 3) Calcul automatique du nombre de qubits d'adresse

In [4]:
nb_pixels = img.size          # ex : 4
nb_addr = int(math.log2(nb_pixels))   # ex : 2 qubits
color = nb_addr               # qubit couleur = le dernier
print("Nombre de qubits de position :", nb_addr)

Nombre de qubits de position : 2


# 4) Construction du circuit FRQI

In [5]:
# Exemple 4 pixels → qubits [0,1] (adresse), 2 (couleur)
qc = QuantumCircuit(nb_addr + 1)

# Étape A : superposition des positions

In [6]:
for q in range(nb_addr):
    qc.h(q)

# Étape B : appliquer RY(θ_i) contrôlée
 selon la position |i>

In [7]:
# ----- PIXEL 0 → adresse |00> -----
qc.x(1); qc.x(0)                            # qubits d'adresse
qc.mcry(2*theta[0], [1,0], color, mode='noancilla')
qc.x(1); qc.x(0)

# ----- PIXEL 1 → adresse |01> -----
qc.x(1)
qc.mcry(2*theta[1], [1,0], color, mode='noancilla')
qc.x(1)

# ----- PIXEL 2 → adresse |10> -----
qc.x(0)
qc.mcry(2*theta[2], [1,0], color, mode='noancilla')
qc.x(0)

# ----- PIXEL 3 → adresse |11> -----
qc.mcry(2*theta[3], [1,0], color, mode='noancilla')

## Affichage du circuit

In [8]:
print(qc.draw())

     ┌───┐┌───┐                                               »
q_0: ┤ H ├┤ X ├────────■───────────■───────────────────────■──»
     ├───┤├───┤        │         ┌─┴─┐                   ┌─┴─┐»
q_1: ┤ H ├┤ X ├────────┼─────────┤ X ├─────────■─────────┤ X ├»
     └───┘└───┘┌───────┴────────┐└───┘┌────────┴────────┐└───┘»
q_2: ──────────┤ U(0.154,0,0,0) ├─────┤ U(-0.154,0,0,0) ├─────»
               └────────────────┘     └─────────────────┘     »
«           ┌───┐                                                           »
«q_0: ──────┤ X ├────────────────────■────────────────■─────────────────────»
«           └───┘       ┌───┐        │         ┌───┐┌─┴─┐                   »
«q_1: ────────■─────────┤ X ├────────┼─────────┤ X ├┤ X ├─────────■─────────»
«     ┌───────┴────────┐└───┘┌───────┴────────┐└───┘└───┘┌────────┴────────┐»
«q_2: ┤ U(0.154,0,0,0) ├─────┤ U(0.616,0,0,0) ├──────────┤ U(-0.616,0,0,0) ├»
«     └────────────────┘     └────────────────┘          └─────────────────┘»
«     